In [50]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from typing import List
from IPython.display import display

# Constants
ALL_STARS_DATASET_LOCATION = './datasets/allstar_player_talent.csv'

In [51]:
def get_head_and_tail(dataframe: pd.DataFrame, head: int, tail: int) -> pd.DataFrame:
    """
    Devuelve un pandas DataFrame con las primer X filas (@param head) y las ultimas Y filas (@param tail).

    @param dataframe: Pandas DataFrame con los datos a utilizar.
    @param head: Int con cantidad de filas a utilizar contando desde el principio del dataframe.
    @param tail: Int con cantidad de filas a utilizar contando desde el final del dataframe

    @return: Pandas DataFrame.
    """
    
    return pd.concat([dataframe.head(n=head), dataframe.tail(n=tail)])

In [52]:
def get_na_from_df(dataframe: pd.DataFrame) -> pd.Series:
    """
    Devuelve un pandas Series con la cantidad de filas vacias por cada fila.

    @param dataframe: Pandas DataFrame con los datos a utilizar.

    @return: Pandas Series.
    """
    
    return dataframe.isna().sum()

In [53]:
# Valor default utilizado para nombar el total de columna por groupby.
DEFAULT_COUNT_COLUMN_NAME = 'count'

def get_distinct_values_for_column(dataframe: pd.DataFrame, column: str, 
    count_name=DEFAULT_COUNT_COLUMN_NAME) -> pd.DataFrame:
    
    """
    Devuelve un dataframe agrupado por una columna y con el total de filas para cada uno de los 
    valores de esa columna.

    @param dataframe: Pandas DataFrame con los datos a utilizar.
    @param column: Str con el nombre de la columna por la que desea agrupar.
    @param count_name: Str con el nombre de la columna con el total de filas. El valor
    defualt es DEFAULT_COUNT_COLUMN_NAME.

    @return: Pandas DataFrame
    """

    return dataframe.groupby(by=column).size().reset_index(name='count')

def get_all_group_by_for_each_column(dataframe: pd.DataFrame, columns: List[str], 
    count_name=DEFAULT_COUNT_COLUMN_NAME) -> List[pd.DataFrame]:

    """
    Devuelve una lista de  dataframes agrupados por una columna y con el total de filas para cada uno de los 
    valores de esa columna.

    @param dataframe: Pandas DataFrame con los datos a utilizar.
    @param column: List[Str] con el nombre de cada una de las
    columnas por las que desea agrupar.
    @param count_name: Str con el nombre de la columna con el total de filas. El valor
    defualt es DEFAULT_COUNT_COLUMN_NAME.

    @return: List of Pandas DataFrame
    """

    list_of_dataframes = []
    for column in columns:
        list_of_dataframes.append(get_distinct_values_for_column(
            dataframe=dataframe, column=column, count_name=count_name))
    
    return list_of_dataframes

def draw_bar_chart(dataframe: pd.DataFrame, x_column: str, y_column: str) -> None:
    """
    Devuelve un gráfico de barras usando el Pandas DataFrame pasado.

    @param dataframe: Pandas DataFrame con los datos a utilizar.
    @param x_column: Str con el nombre de la columna a usar en el eje x.
    @param y_column: Str con el nombre de la columna a user en el eje y.

    @return: None
    """
    
    plt.bar(x_column, y_column, data=dataframe)
    plt.xlabel(x_column),
    plt.xticks(rotation = 90)
    plt.ylabel(y_column)
    plt.title('Cantidad de filas por {}'.format(x_column))
    plt.show()

In [54]:
VALID_SUMMARIZATION_OPERATIONS = ['mean', 'median', 'mode','min','max']
COLUMNS_COLUMN_NAME_DEFAULT = 'Nombre de columna'

def get_summarization_from_data(
    dataframe: pd.DataFrame, columns: List[str], summarization_op: str,
    valid_operations: List[str] = VALID_SUMMARIZATION_OPERATIONS) -> dict:
    """
    Obtiene la sumarización especificada para las columnas pasadas.

    @param dataframe: Pandas DataFrame con los datos a utilizar.
    @param columns: List str de las columnas a sumarizar.
    @param summarization_op: Str con la operación a realizar
    @param valid_operations: List str con las operaciones validas.

    @return: Diccionario con el resultado de la operación
    utilizando como key el nombre de la columna y como value el resultado.
    """

    if summarization_op not in valid_operations:
        raise ValueError('Invalid operation {}. Valid operations are {}'.format(
            summarization_op, valid_operations))

    return getattr(dataframe[columns], summarization_op)().to_dict()


def create_dataframe_with_each_summarization(
    dataframe: pd.DataFrame, columns: List[str], summarization_ops: List[str], 
    columns_column_name: str = COLUMNS_COLUMN_NAME_DEFAULT) -> pd.DataFrame:
    """
    Obtiene las sumarizaciones especificada para las columnas pasadas.

    @param dataframe: Pandas DataFrame con los datos a utilizar.
    @param columns: List str de las columnas a sumarizar.
    @param summarization_ops: Str con las operaciones a realizar
    @param columns_column_name: Str con el nombre a asignarle a la
    columna con los nombres descriptivos de cada columna procesada.

    @return: Pandas DataFrame con los datos de cada sumarizacion por
    columnas.
    """

    initial_column_name = columns_column_name
    new_df = pd.DataFrame(columns, columns=[initial_column_name])

    for operation in summarization_ops:
        new_df[operation] = new_df[initial_column_name].map(
            get_summarization_from_data(
                dataframe=dataframe, 
                columns=columns, 
                summarization_op=operation
            )
        )
    
    return  new_df

def quantile_value_is_valid(value: float) -> bool:
    """
    Valida que el float pasado sea un valor valido de cuantil.

    @param value: float con el número a validar.

    @return: True si el value es valido, sino False.
    """

    if value > 0 and value < 1:
        return True
    return False

def get_quantile_for_data(dataframe: pd.DataFrame, columns: List[str], quantile: float) -> dict:
    """
    Calcula el cuantil especificado para las columnas pasadas.

    @param dataframe: Pandas DataFrame con los datos a utilizar.
    @param columns: List str de las columnas a utilizar para calcular los cuantiles.   
    @param quantile: float con el valor del cuantil a calcular.

    @return: Diccionario que usa como key el nombre de la columna y como
    value el resultado del calculo del cuantil. 
    """
    
    if not quantile_value_is_valid(value=quantile):
        raise ValueError('Quantile value must be between 0 and 1, passed value: {}'.format(quantile))

    return dataframe[columns].quantile(q=quantile).to_dict()

def create_dataframe_with_quantiles(
    dataframe: pd.DataFrame, columns: List[str], quantiles: List[float],
    columns_column_name: str = COLUMNS_COLUMN_NAME_DEFAULT) -> pd.DataFrame:
    """
    Obtiene los cuantiles especificados para las columnas pasadas.

    @param dataframe: Pandas DataFrame con los datos a utilizar.
    @param columns: List str de las columnas a utilizar para calcular los cuantiles.
    @param quantiles: List float con los cuantiles a calcular.
    @param columns_column_name: Str con el nombre a asignarle a la
    columna con los nombres descriptivos de cada columna procesada.

    @return: Pandas DataFrame con los datos de cada cuantil por
    columnas.
    """

    initial_column_name = columns_column_name
    new_df = pd.DataFrame(columns, columns=[initial_column_name])

    for quantile in quantiles:
        new_df['Quantile {}%'.format(quantile * 100)] = new_df[initial_column_name].map(
            get_quantile_for_data(
                dataframe=dataframe,
                columns=columns,
                quantile=quantile
            )
        )
    
    return new_df


In [55]:
all_stars_df = pd.read_csv(ALL_STARS_DATASET_LOCATION, header=0)

## Resumen del dataset
### Conceptos de Baseball
- **(PA) Plate Appearance**: A un jugador se le atribuye una apariencia de placa cada vez que completa un turno de bateo.
- **(IP) Innings pitched**: Son el número de entradas que ha completado un lanzador, medido por el número de bateadores y corredores de base que se eliminan mientras el lanzador está en el montículo de lanzamiento en un juego.
- **(ASG) All Stars Game**: Partido de All Stars.
- **Carreras ofensivas**: Es una estadística de béisbol que mide la cantidad de carreras que un jugador gano o le costó a su equipo en ofensiva en relación con un jugador promedio
- **Carreras defensivas**: Es una estadística de béisbol que mide la cantidad de carreras que un jugador salvó o le costó a su equipo en defensa en relación con un jugador promedio

### Descripción de cada columna

Columnas del dataset All Players Talent
| Columna  | Descripción  |
|---|---|
| bbref_ID  | ID del jugado basado en [Baseball-Reference.com](https://www.baseball-reference.com/). |
| yearID  | La temporada  |
| gameNum  | Orden del partido dependiendo de la temporada.  |
| gameId  | ID del partido basado en [Baseball-Reference.com](https://www.baseball-reference.com/).   |
| lgID  |  ID de la liga del equipo de All Stars. |
| startingPos  | Posición en la que juego el jugador. [Convención usada.](https://en.wikipedia.org/wiki/Baseball_positions#:~:text=Each%20position%20conventionally%20has%20an,and%209%20(right%20fielder).)  |
| OFF600  | Estimación de talento ofensivo, en carreras por encima del promedio de la liga por cada 600 *PA*. [Link para más información sobre Offensive Stats](https://www.rookieroad.com/baseball/statistics/offensive-stats/). |
| DEF600  | Estimación del talento de fildeo, en carreras por encima del promedio de la liga por cada 600 *PA*. [Link para más información sobre Deffensive Stats](https://www.rookieroad.com/baseball/statistics/defensive-stats/). |
| PITCH200  | Estimación del talento de lanzamiento, en carreras por encima del promedio de la liga por 200 *IP*. [Link para más información sobre Pitcher Stats](https://www.rookieroad.com/baseball/statistics/pitcher-stats/). |
| asg_PA  | Número de PA en el partido.  |
| asg_IP  | Número de IP en el partido.  |
| OFFper9innASG | Carreras ofensivas esperadas añadidas por encima del promedio (de talento) basadas en PA en ASG, escaladas a un juego de 9 entradas  |
| DEFper9innASG | Carreras defensivas esperadas añadidas por encima del promedio (de talento) basadas en PA en ASG, escaladas a un juego de 9 entradas  |
| PITper9innASG | Carreras de lanzamiento esperadas agregadas por encima del promedio (de talento) basadas en IP en ASG, escaladas a un juego de 9 entradas  |
| TOTper9innASG | Carreras esperadas añadidas por encima del promedio (de talento) basadas en PA/IP en ASG, escaladas a un juego de 9 entradas  |

In [ ]:
get_head_and_tail(all_stars_df, head=10, tail=10)

### Cantidad de nulos por columna

In [ ]:
get_na_from_df(dataframe=all_stars_df)

## Análisis de variables cualitativas
### Columnas con variables cualitativas

- yearID
- gameNum 
- gameID 
- lgID
- startingPos

In [ ]:
list_of_qualitative_columns = ['yearID', 'gameNum', 'lgID', 'startingPos']

group_by_dfs = get_all_group_by_for_each_column(dataframe=all_stars_df, columns=list_of_qualitative_columns)

for column, df in zip(list_of_qualitative_columns, group_by_dfs):
    print(column)
    display(df)
    draw_bar_chart(dataframe=df, x_column=column, y_column=DEFAULT_COUNT_COLUMN_NAME)

### ¿Es posible realizar este gráfico con un histograma?

## Análisis de variables cuantitatuvas
### Columnas con variables cuantitativas

- OFF600
- DEF600 
- PITCH200 
- asg_PA
- asg_IP
- OFFper9innASG
- DEFper9innASG
- PITper9innASG
- TOTper9innASG

In [ ]:
list_of_quantitative_columns = ['OFF600', 'DEF600', 'PITCH200', 'asg_PA', 'asg_IP', 'OFFper9innASG', 'DEFper9innASG', 'PITper9innASG', 'TOTper9innASG']
summarization_operations = ['mean', 'median', 'mode','min','max']


summarized_df = create_dataframe_with_each_summarization(
    dataframe=all_stars_df,
    columns=list_of_quantitative_columns,
    summarization_ops=summarization_operations)

quantiles_df = create_dataframe_with_quantiles(
    dataframe=all_stars_df, 
    columns=list_of_quantitative_columns, 
    quantiles=[0.25, 0.75])

pd.merge(left=summarized_df, right=quantiles_df, left_on='Nombre de columna', right_on='Nombre de columna')


## Correlación de las variables

In [ ]:
all_stars_only_quantitative_variables = all_stars_df[list_of_quantitative_columns]
sns.heatmap(data=all_stars_only_quantitative_variables.corr(method='pearson'), cmap='Reds')

## Scatterploat Matrix

In [ ]:
sns.pairplot(data=all_stars_only_quantitative_variables)

## Analisis de un subconjunto de variables 

### Subconjunto de variables cuantitativas elegidas

- asg_PA
- OFF600
- DEF600 

### Histogramas

In [56]:
list_of_selected_columns = ['OFF600','DEF600','asg_PA']
all_stars_only_selected_variables = all_stars_only_quantitative_variables[list_of_selected_columns]


all_stars_only_selected_variables['OFF600'].hist(alpha = 0.5, bins =20, color = 'green')
all_stars_only_selected_variables['DEF600'].hist(alpha = 0.5, bins =20, color = 'red')


In [57]:
all_stars_only_selected_variables['asg_PA'].hist(alpha = 0.5, bins =7)

### Grafico de violín

In [58]:
plt.title('Grafico de violin para las variables seleccionadas')
plt.violinplot(all_stars_only_selected_variables, showmeans = True)
plt.xlabel("Variables OFF600 , DEF600 y asg_PA")
plt.ylabel("Valores")
plt.grid(True)
plt.show()

### Boxplots

In [59]:
sns.boxplot(data = all_stars_only_selected_variables).set(title="Boxplot variables seleccionadas", xlabel = "variables", ylabel = "Valores")

### Graficos de densidad

In [60]:
all_stars_only_selected_variables[['OFF600','DEF600']].plot.kde(title = "Densidad de valores")

In [61]:
all_stars_only_selected_variables['asg_PA'].plot.kde(title = "Densidad de asg_PA")

### Grafico a elección

In [62]:
import plotly.express as px
fig = px.line(all_stars_only_selected_variables)
fig.show()

### Concluciones